# Introdução

In [ ]:
import os
import re
import time
import json
import folium
import random
import requests
import numpy as np
import pandas as pd
import seaborn as sns
import geopandas as gpd
from folium import plugins
from osgeo import gdal, osr
from bs4 import BeautifulSoup
from tqdm.notebook import trange, tqdm

<br>

# Join Data

In [ ]:
# Lê o arquivo csv com o nome dos municípios
df = pd.read_csv(
    'https://raw.githubusercontent.com/michelmetran/sp_cfb/main/data/tabs/tab_municipio_ctr.csv',
)

# Lê o arquivo csv com o nome dos municípios
gdf = gpd.read_file(
    'https://raw.githubusercontent.com/michelmetran/sp/main/data/shps/sp_250k_wgs84.geojson',
)
gdf.drop(['municipio_nome'], axis=1, inplace=True)
gdf['id_municipio'] = gdf['id_municipio'].astype(int)
gdf['geometry'] = gdf.simplify(0.0015)

# Merge
gdf = gdf.merge(
    df,
    on='id_municipio',
    how='left'
)

# Results
gdf.head()

<br>

## Create pop up's column

In [ ]:
# Add Field
def popup_html(row):
    tel1 = str(row['telefone_1']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    tel2 = str(row['telefone_2']).replace('-', '').replace(')', '').replace('(', '+55').replace(' ', '')
    
    html = """
    <div>
    <p><b>{}</b> pertence à:
    <h4><b>{} - {}</b></h4></p>
    <p><b>Sede:</b><br>
    {}{}{}
    {}
    {}
    {}</p>
    <p><b>Contatos:</b>
    {}
    {}
    {}</p>
    <p><b>Diretor:</b><br>
    {}</p>
    </div>
    """.format(
        '' if pd.isnull(row['municipio_nome']) else '{}'.format(row['municipio_nome']),
        '' if pd.isnull(row['id_ctr'])         else '{}'.format(row['id_ctr']),
        '' if pd.isnull(row['ctr_nome'])       else '{}'.format(row['ctr_nome']),
        '' if pd.isnull(row['endereco'])       else '{}'.format(row['endereco']),
        '' if pd.isnull(row['numero'])         else ', {}'.format(row['numero']),
        '' if pd.isnull(row['complemento'])    else ' - {}'.format(row['complemento']),
        '' if pd.isnull(row['bairro'])         else '<br>Bairro: {}'.format(row['bairro']),
        '' if pd.isnull(row['municipio_sede']) else '<br>Município: {}'.format(row['municipio_sede']),
        '' if pd.isnull(row['cep'])            else '<br>CEP: {}'.format(row['cep']),
        '' if pd.isnull(row['telefone_1'])     else '<br>Telefone: <a href="tel:{}">{}</a>'.format(tel1, row['telefone_1']),
        '' if pd.isnull(row['telefone_2'])     else ' / <a href="tel:{}">{}</a>'.format(tel2, row['telefone_2']),
        '' if pd.isnull(row['email'])          else '<br>E-mail: <a href="mailto:{}">{}</a>'.format(row['email'], row['email']),
        '' if pd.isnull(row['diretor'])        else '{}'.format(row['diretor']),
    )
    
    html = html.replace('\n','')
    html = re.sub('\s\s+' , ' ', html) # Remove Espaços no meio
    html = html.strip()
    return html

In [ ]:
# Calculate PopUps
gdf['popup'] = gdf.apply(popup_html, axis=1)

<br>

## Adjust Table

In [ ]:
# Delete Columns
print(gdf.columns)
gdf.drop([
    'id_municipio',
    'endereco', 'numero', 'bairro',
    'municipio_sede', 'cep', 'telefone_1', 'telefone_2',
    'complemento', 'email',], axis=1, inplace=True)


In [ ]:
# Save geojson
gdf.to_file(
    os.path.join('data', 'shps', 'sp_ctr.geojson'),
    driver='GeoJSON',
    encoding='utf-8'
)

# Results
gdf.head()

## Get Centroid

In [ ]:
def get_centroid(gdf):
    gdf['apenasparacentroid'] = 35
    gdf_dissolve = gdf.dissolve(by='apenasparacentroid')
    gdf_centroid = gdf_dissolve.representative_point()
    gdf = gdf.drop('apenasparacentroid', axis=1)
    return [float(gdf_centroid.y), float(gdf_centroid.x)]

In [ ]:
list_centroid = get_centroid(gdf)
list_centroid

## Folium

In [ ]:
def map_bomb(input_geojson, bbox):
    gdf = gpd.read_file(input_geojson)
    
    # Column with category
    col_categories = 'id_ctr'
    
    # Set palette
    palette_polygon = 'Paired'

    # Get list of unique values
    categories = set(gdf[col_categories])
    categories = list(categories)
    categories.sort()

    # See the palette chosed
    pal = sns.color_palette(palette_polygon, n_colors=len(categories))

    # Set dictionary
    color_polygon = dict(zip(categories, pal.as_hex()))
    #color_polygon['Agência Ambiental de Ribeirão Preto'] = '#e699ff'

    # Create Map
    m = folium.Map(
        location=[-22.545968889465207, -49.56185387118866],
        zoom_start=6,
        min_zoom=6,
        max_zoom=11,
        max_bounds=True,
        min_lon = bbox['min_lon']*(101/100),
        max_lon = bbox['max_lon']*(99/100),        
        min_lat = bbox['min_lat']*(101/100),
        max_lat = bbox['max_lat']*(99/100),
        #zoom_delta=0.1,
    )
    folium.GeoJson(
        gdf,
        name='CETESB',
        smooth_factor=1.0,
        zoom_on_click=False,
        embed=False,
        style_function=lambda x: {
            'fillColor': color_polygon[x['properties'][col_categories]],
            'color': color_polygon[x['properties'][col_categories]],
            'weight': 1,
            'fillOpacity': 0.3,
        },
        highlight_function=lambda x: {
            'weight': 3,
            'fillOpacity': 0.6,
        },
        tooltip=folium.features.GeoJsonTooltip(
            fields=['municipio_nome', 'id_ctr'],
            aliases=['Munícipio', 'CTR'],
            sticky=True,
            opacity=0.9,
            direction='right',
        ),
        popup=folium.GeoJsonPopup(
            ['popup'],
            parse_html=False,
            max_width='400',
            show=False,
            labels=False,
            sticky=True,            
        )        
    ).add_to(m)

    # Plugins
    m.fit_bounds(m.get_bounds())
    plugins.Fullscreen(
        position='topleft',
        title='Clique para Maximizar',
        title_cancel='Mininizar',
    ).add_to(m)
    return m

In [ ]:
# Map without Bounds
bbox = {
    'max_lat': 0,
    'max_lon': 0,
    'min_lat': 0,
    'min_lon': 0,
}
m = map_bomb(os.path.join('data', 'shps', 'sp_ctr.geojson'), bbox)

# Map with Bounds
bbox = {
    'max_lat': m.get_bounds()[1][0],
    'min_lat': m.get_bounds()[0][0],
    'max_lon': m.get_bounds()[1][1],
    'min_lon': m.get_bounds()[0][1],
}
m = map_bomb(os.path.join('data', 'shps', 'sp_ctr.geojson'), bbox)

# Results
print(bbox)
m

<br>

# Save Map

In [ ]:
os.makedirs('maps', exist_ok=True)
m.save(os.path.join('maps', 'cfb_map.html'))
m.save(os.path.join('..', '..', '..', 'case_django', 'divadmin', 'templates', 'cfb_map.html'))